In [1]:
import xarray as xr
import xroms
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# How to select data

The [input/output](io.ipynb) notebook demonstrates how to load in data, but now how to select and slice it apart? Much of this is accomplished with the `sel` and `isel` methods in `xarray`, which are demonstrated in detail in this notebook. 
* Use `sel` to subdivide a Dataset or DataArray by dimension values; the best example of this for ROMS output is selecting certain time using a string representation of a datetime.

    >ds.salt.sel(ocean_time='2010-1-1 12:00')
    
    >ds.salt.sel(ocean_time=slice('2010-1-1', '2010-2-1'))

* Use `isel` to subdivide a Dataset or DataArray by dimension indices:

    >ds.salt.isel(eta_rho=20, xi_rho=100)
    
    >ds.salt.isel(eta_rho=slice(20,100,10), xi_rho=slice(None,None,5))



## `cf-xarray`

`xroms` includes the `cf-xarray` accessor, which allows you to use xarray `sel` and `isel` commands for a DataArray without needing to input the exact grid – just the axes.

With `xarray` alone:

    >ds.salt.isel(xi_rho=20, eta_rho=10, s_rho=20, ocean_time=10)

With `cf-xarray` accessor:

    >ds.salt.cf.isel(X=20, Y=10, Z=20, T=10)
    
and get the same thing back. Same for `sel`. The T, Z, Y, X names can be mixed and matched with the actual dimension names. Some of the attribute wrangling in `xroms` is dedicated to making sure that `cf-xarray` can always identify dimensions and coordinates for DataArrays.

You can always check what `cf-xarray` understands about a Dataset or DataArray with

    ds.salt.cf.describe()

### Load in data

More information at in [input/output notebook](io.ipynb)

In [23]:
tod = pd.Timestamp.today()
url = f"https://opendap.co-ops.nos.noaa.gov/thredds/fileServer/NOAA/CIOFS/MODELS/{tod.year}/{str(tod.month).zfill(2)}/{str(tod.day).zfill(2)}/nos.ciofs.fields.n001.{str(tod.date()).replace('-','')}.t00z.nc"

In [25]:
!wget $url

--2023-05-22 17:28:21--  https://opendap.co-ops.nos.noaa.gov/thredds/fileServer/NOAA/CIOFS/MODELS/2023/05/22/nos.ciofs.fields.n001.20230522.t00z.nc
Resolving opendap.co-ops.nos.noaa.gov (opendap.co-ops.nos.noaa.gov)... 137.75.111.179, 137.75.111.136
Connecting to opendap.co-ops.nos.noaa.gov (opendap.co-ops.nos.noaa.gov)|137.75.111.179|:443... connected.
HTTP request sent, awaiting response... 200 
Length: 587516760 (560M) [application/x-netcdf]
Saving to: ‘nos.ciofs.fields.n001.20230522.t00z.nc’

ciofs.fields.n001.2   5%[>                   ]  32.99M   147KB/s    eta 65m 30s^C


In [17]:
ls

calc-Copy1.ipynb           io-Copy1.ipynb
calc.ipynb                 plotting-Copy1.ipynb
calc.md                    plotting.ipynb
interpolation-Copy1.ipynb  select_data-Copy1.ipynb
interpolation.ipynb        select_data.ipynb


In [2]:
# Helpful to have local file for this
url = 'http://barataria.tamu.edu:8080/thredds/fileServer/forecast_latest/txla2_his_a_latest.nc'
fname = url.split('/')[-1]
chunks = {'ocean_time': 1}

import os
if os.path.exists(fname):
    ds = xroms.open_netcdf(fname, chunks=chunks)
else:
    from subprocess import call
    call(['wget', url])
    ds = xroms.open_netcdf(fname, chunks=chunks)
    
# Alternatively, use file through thredds:
# loc = 'http://barataria.tamu.edu:8080/thredds/dodsC/forecast_latest/txla2_his_f_latest.nc'
# chunks = {'ocean_time':1}
# ds = xr.open_dataset(loc, chunks=chunks)

--2023-05-22 13:27:01--  http://barataria.tamu.edu:8080/thredds/fileServer/forecast_latest/txla2_his_a_latest.nc
Resolving barataria.tamu.edu (barataria.tamu.edu)... failed: nodename nor servname provided, or not known.
wget: unable to resolve host address ‘barataria.tamu.edu’


FileNotFoundError: [Errno 2] No such file or directory: b'/Users/kthyng/projects/xroms/examples/txla2_his_a_latest.nc'

## Select

### Surface layer slice

The surface in ROMS is given by the last index in the vertical dimension. The easiest way to access this is by indexing into `s_rho`. While normally it is better to access coordinates through keywords to be human-readable, it's not easy to tell what value of `s_rho` gives the surface. In this instance, it's easier to just go by index.

In [ ]:
ds.salt.isel(s_rho=-1);

# or with cf-xarray accessor:
ds.salt.cf.isel(Z=-1);

### x/y index slice

For a curvilinear ROMS grid, selecting by the dimensions `xi_rho` or `eta_rho` (or for whichever is the relevant grid) is not very meaningful because they are given by index. Thus the following is possible to get a slice along the index, but it cannot be used to find a slice based on the lon/lat values. For the eta and xi grids, `sel` is equivalent to `isel`.

In [ ]:
ds.temp.sel(xi_rho=20);

ds.temp.cf.sel(X=20);  # same with cf-xarray accessor

### Single time

Find the forecast model output available that is closest to now. Note that the `method` keyword argument is not necessary if the desired date/time is exactly a model output time. You can daisy-chain together different `sel` and `isel` calls.

In [ ]:
now = pd.Timestamp.today()

In [ ]:
ds.salt.isel(s_rho=-1).sel(ocean_time=now, method='nearest');

ds.salt.cf.isel(Z=-1).cf.sel(T=now, method='nearest'); 

### Range of time

In [ ]:
ds.salt.sel(ocean_time=slice(now, now+pd.Timedelta('2 days')));

ds.salt.cf.sel(T=slice(now, now+pd.Timedelta('2 days'))); 

### Select region

Select a boxed region by min/max lon and lat values.

In [ ]:
# want model output only within the box defined by these lat/lon values
lon = np.array([-97, -96])
lat = np.array([28, 29])

In [ ]:
# this condition defines the region of interest
box = ((lon[0] < ds.lon_rho) & (ds.lon_rho < lon[1]) & (lat[0] < ds.lat_rho) & (ds.lat_rho < lat[1])).compute()

Plot the model output in the box at the surface

In [ ]:
dss = ds.where(box).salt.cf.isel(Z=-1, T=0)
dss.cf.plot(x='longitude', y='latitude')

If you don't need the rest of the model output, you can drop it by using `drop=True` in the `where` call.

In [ ]:
dss = ds.where(box, drop=True).salt.cf.isel(Z=-1, T=0)
dss.cf.plot(x='longitude', y='latitude')

Can calculate a metric within the box:

In [ ]:
dss.mean().values

### Subset model output

Subset Dataset of model output such that subsetted domain is as if the simulation was run on that size grid. That is, the rho grid is 1 larger than the psi grid in each of xi and eta.

In [ ]:
xroms.subset(ds, X=slice(20,40), Y=slice(50,100))

In [ ]:
ds.xroms.subset(X=slice(20,40), Y=slice(50,100))  # with accessor

### Find nearest in lon/lat

This matters for a curvilinear grid.

Can't use `sel` because it will only search in one dimension for the nearest value and the dimensions are indices which are not necessarily geographic distance. Instead need to use a search for distance and use that for the `where` condition from the previous example. This functionality has been wrapped into `xroms.sel2d` (and its partner function `xroms.argsel2d`).

In [ ]:
lon0, lat0 = -96, 27
saltsel = ds.salt.xroms.sel2d(lon0, lat0)

Or, if you instead want the indices of the nearest grid node returned, you can call `argsel2d`:

In [ ]:
ds.salt.xroms.argsel2d(lon0, lat0)

Check this function, just to be sure:

In [ ]:
dl = 0.05
box = (ds.lon_rho>lon0-dl) & (ds.lon_rho<lon0+dl) & (ds.lat_rho>lat0-dl) & (ds.lat_rho<lat0+dl)
dss = ds.where(box).salt.cf.isel(T=0, Z=-1)

vmin = dss.min().values
vmax = dss.max().values

dss.plot(x='lon_rho', y='lat_rho')
plt.scatter(lon0, lat0, c=saltsel.cf.isel(Z=-1, T=0), s=200, edgecolor='k', vmin=vmin, vmax=vmax)
plt.xlim(lon0-dl,lon0+dl)
plt.ylim(lat0-dl, lat0+dl)